<a href="https://colab.research.google.com/github/tavakohr/BasicFunctions/blob/master/RealEstate_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R Project to predic the sale price

In [2]:
%load_ext rpy2.ipython
from google.colab import files

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


##1.1 load spreadsheet for training 

In [3]:
Spreadsheet = files.upload()

Saving Spreadsheet-5.xlsx to Spreadsheet-5.xlsx


##1.2  load HPI index

In [4]:
HPI = files.upload()

Saving HPI.csv to HPI.csv


## 1.3 load Sale to Active index

In [5]:
SaleToActive= files.upload()

Saving SaleToActive.csv to SaleToActive.csv


In [10]:
%%R
if (!require(lme4)) install.packages('lme4')
if (!require(caret)) install.packages('caret')


R[write to console]: Loading required package: caret

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘plyr’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/numDeriv_2016.8-1.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 76053 bytes (74 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

## 1.4 load requirements

In [9]:
%%R
library(readxl)
library(tidyverse)
library(lme4)
library(caret)
library(lubridate)
library(rpart)
library(e1071)
library(readxl)
library(fastDummies)

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘Matrix’


R[write to console]: The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


R[write to console]: Error in library(caret) : there is no package called ‘caret’

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
R[write to console]: 
 
R[write to console]:  there is no package called ‘lme4’




Error in library(caret) : there is no package called ‘caret’


RInterpreterError: ignored

## 1.5 Create Calibration plot function

In [ ]:
%%R
myrank<-function(longvector){
  longvector_orderd<-sort(longvector)
  bin=length(longvector_orderd)/10
  rank=floor((row_number(longvector_orderd)-1)/bin)
  df=cbind.data.frame(rank=rank ,longvector)
  return(df)
  
}


caliberPlot<-function(observed, prdedict){
  
  
  calib_pred<-myrank(prdedict) 
  calib_pred<-cbind.data.frame(calib_pred,observed=observed)
  
  
  gdf=calib_pred %>% group_by(rank) %>% summarise(mean_predict=mean(longvector))
  gdf2=calib_pred %>% group_by(rank) %>% summarise(mean_observed=mean(observed))
  gdf3<-cbind.data.frame(gdf,gdf2[,2])
  
  ggplot(data=gdf3,aes(x=mean_predict,y=mean_observed))+geom_line()+geom_abline()
}

#2 Data Cleaning

## 2.1- import and clean sale records ################################

In [ ]:
%%R
a_imported<-read_excel('Spreadsheet-5.xlsx', sheet='Spreadsheet-5')
a_imported<-a_imported %>%   rename( List.Date ='List Date' )
a_imported<-a_imported %>%   rename( Tax ='Gross Taxes' ) %>% 
  rename( Tot.BR ='Tot BR' ) %>% rename( Tot.Baths ='Tot Baths' ) %>%  rename(  Kitchens  ='#Kitchens' ) %>% 
  rename(  Distance.Pub  ='Distance to Pub/Rapid Tr' ) %>% 
  rename(  Lot  ='Lot Sz (Sq.Ft.)' ) %>%   rename(  Style.of.Home  ='Style of Home' ) %>%   rename(  Yr.Blt  ='Yr Blt' ) %>% 
  rename(  ML..  ='ML #' ) %>%   rename( Approx.Yr.of.Renos.Addns='Approx.Yr of Renos/Addns' ) %>% 
  rename( S.A='S/A' ) %>% 
  rename( ML='ML..' ) %>% 
  filter(TypeDwel=='HOUSE', Tax>0)

a_imported<-a_imported %>% filter(Address!='' )
#a_imported$List.Date <- mdy(a_imported$ List.Date)
a_imported$Price=as.numeric(gsub('\\$|,', '', a_imported$Price))


In [ ]:
%%R
names(a_imported)

### 2.1.1 create city column with acronym

In [ ]:
%%R
#str(a_imported)
table(a_imported$Distance.Pub)

a_imported <- a_imported %>% mutate(SaletoAcrive_YM=paste(year(List.Date ), month(List.Date), sep='')) %>%  rename( AskingPrice='ListPrice' ) %>%
  rename( SalePrice='Price' )    

a_imported <- a_imported %>% mutate(HPI_YM=paste(year(List.Date ), month(List.Date), sep='')) %>% filter(Tax>0) %>% 
  mutate(city=if_else(City=='Port Coquitlam','VPQ','VCQ'))

## 2.2  clean sale and HPI indicators  


In [ ]:
%%R
# for now I just have the coquitlam's records , I assume the indicators in both coquitlam and POCO changed in the same direction

SaletoAcrive<-read.csv('SaleToActive.csv' )
HPI<-read.csv('HPI.csv' )
 
SaletoAcrive$Date<-as.Date( SaletoAcrive$Date)
HPI$Date<-as.Date(HPI$Date)
#str(SaletoAcrive)
#str(HPI)
# each indicator is a predictive variable for the next month not the same one:

SaletoAcrive<-SaletoAcrive %>% mutate(SaletoAcrive_YM=paste(year(Date %m+% months(1)), month(Date %m+% months(1)), sep='') )

HPI<-HPI %>% mutate(HPI_YM=paste(year(Date %m+% months(1)), month(Date %m+% months(1)), sep=''))

#Left join main data sets to the indicator tables:
#str(a_imported)
a_imported_1<-left_join (a_imported,SaletoAcrive, by=c('city','SaletoAcrive_YM')) %>% dplyr::select(-Date)
a_imported_2<-left_join (a_imported_1,HPI,    by=c('city','HPI_YM'))



db<-rbind.data.frame(a_imported_2 )


db<-db %>% filter(TypeDwel=='HOUSE' )

db$Lot <- as.numeric(gsub('\\$|,', '', db$Lot))
db$TotFlArea <- as.numeric(gsub('\\$|,', '', db$TotFlArea))
db$S.A<-as.factor(db$S.A)
table(db$S.A)
#str(db)
db<-db %>%   filter(Lot>2000  , SalePrice <2500000 ,SalePrice >990000)
db$month <- month(db$List.Date)
db$year <- year(db$List.Date)


db<-db %>% mutate(reno=ifelse(is.na(Approx.Yr.of.Renos.Addns),
                              0,ifelse(Approx.Yr.of.Renos.Addns>2016,3,ifelse(Approx.Yr.of.Renos.Addns<=2016 &Approx.Yr.of.Renos.Addns>2010 ,2,1)) ))
table(db$reno)

db<-db %>% filter(Lot>0,Age<70)

#str(db) 

db<-db %>% select(ML,SalePrice,AskingPrice,Tot.BR,Tot.Baths, Age,Kitchens,TotFlArea,Lot,S.A,month,year, reno,List.Date,SaleToActive ,HPI,Tax)
#str(db)


names(db)

In [ ]:
%%R
head(db)

## 2.3 apply the cooks distance to remove high leverage points  


In [ ]:
%%R
db_fit<-glm(SalePrice~AskingPrice  ,
                  family = gaussian(link = "identity"),
                  data=db)
High_leverage<-db[cooks.distance(db_fit)>1,]
High_leverage



###2.3.1 Exclude cases with huge abnormality in asking and sale price

In [ ]:
%%R
db<-db[cooks.distance(db_fit)<=1,]


In [ ]:
%%R
dim(db)

## 2.4 exclude the outlires

### 2.4.1 Price per bedroom

In [ ]:
%%R
ggplot(data=db,aes(x=as.factor(Tot.BR),y=SalePrice))+geom_boxplot()

In [ ]:
%%R
db<-db[db$Tot.BR<10,]
db<-db[db$Tot.BR>0,]

### 2.4.2 Price per lot per building age

In [ ]:
%%R
db<-db %>% mutate(Agegroup=ifelse(Age <15 ,1,ifelse(Age<40,2,3)))
table(db$Agegroup)
ggplot(data=db,aes(x=Lot,y=SalePrice))+geom_point(aes(colour=as.factor(Agegroup)))